In [5]:
import os

from dotenv import load_dotenv
from etg import GuestRoom

load_dotenv()

# ETG API Credentials
ETG_KEY_ID = os.environ["ETG_KEY_ID"]
ETG_API_KEY = os.environ["ETG_API_KEY"]
ETG_REQUEST_TIMEOUT = 30.0

# Search Parameters
CITY = "Москва"

CHECKIN_DATE = "2026-01-30"
CHECKOUT_DATE = "2026-02-04"

CURRENCY = "RUB"
LANGUAGE = "ru"
RESIDENCY = "RU"

GUESTS: list[GuestRoom] = [{"adults": 2, "children": [4, 2]}]
LIMIT = 1000

# User preferences for AI
USER_PREFERENCES = "Обязательно две комнаты и две кровати. Хорошие отзывы. Чистота"

# Filters
MIN_PRICE: float | None = 3000.0  # None = no minimum
MAX_PRICE: float | None = 20000.0  # None = no maximum

In [6]:
from etg import ETGClient, GuestRoom, Hotel, HotelContent

client = ETGClient(ETG_KEY_ID, ETG_API_KEY, timeout=ETG_REQUEST_TIMEOUT)


def ostrovok_url(
    hotel_id: str,
    hid: int,
    checkin: str,
    checkout: str,
    guests: list[GuestRoom],
    region_id: int,
    country_slug: str = "russia",
    city_slug: str = "moscow",
) -> str:
    """Generate Ostrovok booking URL for a hotel.
    
    Args:
        hotel_id: Hotel string ID (e.g., 'novum_hotel_aldea_berlin_centrum')
        hid: Hotel numeric ID
        checkin: Check-in date (YYYY-MM-DD)
        checkout: Check-out date (YYYY-MM-DD)
        guests: List of room configurations
        region_id: Region ID for the search
        country_slug: Country URL slug (default: 'russia')
        city_slug: City URL slug (default: 'moscow')
    
    Returns:
        Full Ostrovok URL for the hotel
    """
    # Convert dates from YYYY-MM-DD to DD.MM.YYYY
    ci_parts = checkin.split("-")
    co_parts = checkout.split("-")
    dates = f"{ci_parts[2]}.{ci_parts[1]}.{ci_parts[0]}-{co_parts[2]}.{co_parts[1]}.{co_parts[0]}"
    
    # Calculate total guests
    total_guests = sum(
        room.get("adults", 0) + len(room.get("children", []))
        for room in guests
    )
    
    base = f"https://ostrovok.ru/hotel/{country_slug}/{city_slug}"
    return f"{base}/mid{hid}/{hotel_id}/?dates={dates}&guests={total_guests}&q={region_id}"

In [7]:
async def find_region_id(client: ETGClient, city_name: str, language: str) -> int | None:
    """Find region ID for a city name."""
    print(f"Looking up region ID for '{city_name}'...")
    regions = await client.suggest_region(city_name, language)

    if not regions:
        print(f"  No regions found for '{city_name}'")
        return None

    # Only accept City type
    for region in regions:
        if region["type"] == "City":
            region_id = region["id"]
            print(f"  Found: {region['name']} ({region.get('country_code', '')}), region_id={region_id}")
            return region_id

    # No city found - show available options
    print(f"  No city found. Available regions:")
    for r in regions[:5]:
        print(f"    - {r['name']} (type: {r['type']}, id: {r['id']})")

    return None

In [8]:
# Find region by city name.
# ETG API requires region_id for hotel search, so we first
# lookup the region ID via suggest_region by city name.
region_id = await find_region_id(client, CITY, LANGUAGE)
if not region_id:
    raise ValueError(f"Could not find region for '{CITY}'")

print(f"\nSearching hotels in {CITY}...")
print(f"  Dates: {CHECKIN_DATE} to {CHECKOUT_DATE}")
print(f"  Currency: {CURRENCY}, Limit: {LIMIT}")

Looking up region ID for 'Москва'...
  Found: Москва (RU), region_id=2395

Searching hotels in Москва...
  Dates: 2026-01-30 to 2026-02-04
  Currency: RUB, Limit: 1000


In [9]:
import pandas as pd

from services import search_hotels

# Search available hotels in the region with given parameters.
# Filters by price range if MIN_PRICE/MAX_PRICE are set.
# Returns short hotel info: id, hid, and rates (room name, price, meal).
# Full content (name, address, amenities) is fetched separately via hotel content API.
search_result = await search_hotels(
    client=client,
    region_id=region_id,
    checkin=CHECKIN_DATE,
    checkout=CHECKOUT_DATE,
    residency=RESIDENCY,
    guests=GUESTS,
    currency=CURRENCY,
    language=LANGUAGE,
    hotels_limit=LIMIT,
    min_price=MIN_PRICE,
    max_price=MAX_PRICE,
)

hotels = search_result["hotels"]
total_available = search_result["total_available"]
total_after_filter = search_result["total_after_filter"]

In [10]:
if not hotels:
    print("No hotels found for the given criteria.")
    df_hotels = pd.DataFrame()
else:
    print(f"Found {total_after_filter} hotels after price filter (total available: {total_available})\n")

    # Create DataFrame
    hotels_data = []
    for hotel in hotels:
        rates = hotel.get("rates", [])
        if rates:
            first_rate = rates[0]
            payment_types = first_rate.get("payment_options", {}).get("payment_types", [])
            price = float(payment_types[0].get("show_amount", 0)) if payment_types else None
            room_name = first_rate.get("room_name", "")
            meal = first_rate.get("meal", "")
        else:
            price = None
            room_name = ""
            meal = ""

        hotels_data.append({
            "hotel_id": hotel["id"],
            "hid": hotel["hid"],
            "price": price,
            "currency": CURRENCY,
            "room": room_name[:40] if room_name else "",
            "meal": meal,
        })

    df_hotels = pd.DataFrame(hotels_data)
    df_hotels = df_hotels.sort_values("price", ascending=True).reset_index(drop=True)
    df_hotels.index += 1  # Start from 1

    # Display settings
    pd.set_option("display.max_colwidth", 50)
    pd.set_option("display.max_rows", 100)

df_hotels

Found 205 hotels after price filter (total available: 231)



,hotel_id,hid,price,currency,room,meal
1,vozle_parka_botanicheskiy_sad_flat,11282356,16671.0,RUB,Четырёхместные апартаменты Standard с ба,nomeal
2,minihotel_ladomir_na_yauze,8662675,16729.0,RUB,Двухместный номер Семейный Улучшенный 2,nomeal
3,brusnika_schelkovskaya_minihotel,8481957,17489.0,RUB,Четырёхместный номер Комфорт с диван-кро,nomeal
4,zvezda_hotel_6,10437016,17500.0,RUB,Трёхместный номер Comfort (дополнительна,nomeal
5,knokey_tekstilschiki_apartments,13313968,17687.0,RUB,Апартаменты (питание для детей не включе,nomeal
...,...,...,...,...,...,...
201,palmira_garden_hotel_spa_palmira_garden,9291179,96600.0,RUB,Двухместный полулюкс (двуспальная кроват,nomeal
202,otel_radisson_slavyanskaya_and_business_centre,7467357,97600.0,RUB,Двухместный люкс с 2 комнатами (двуспаль,nomeal
203,doubletree_by_hilton_moscow_vnukovo_airport,8848335,97750.0,RUB,Двухместный номер Deluxe с красивым видо,nomeal
204,grand_revival_hotel,10004948,98647.0,RUB,Четырёхместный номер Kомфорт с джакузи (,nomeal


In [11]:
from services import batch_get_content

hotel_hids = [h["hid"] for h in hotels]

print(f"[batch_get_content_start] Загрузка контента для {len(hotel_hids)} отелей...")
content_result = await batch_get_content(client, hotel_hids, LANGUAGE)
content_map = content_result["content"]
print(
    f"[batch_get_content_done] Загружен контент для {content_result['total_loaded']} "
    f"из {content_result['total_requested']} отелей ({content_result['total_batches']} батчей)"
)

[batch_get_content_start] Загрузка контента для 205 отелей...
[batch_get_content_done] Загружен контент для 205 из 205 отелей (3 батчей)


In [12]:
content_data = []
for hid, content in content_map.items():
    content_data.append({
        "hid": hid,
        "name": content.get("name", "")[:40],
        "stars": content.get("star_rating", 0),
        "kind": content.get("kind", ""),
        "address": content.get("address", "")[:50],
        "latitude": content.get("latitude"),
        "longitude": content.get("longitude"),
        "check_in": content.get("check_in_time", ""),
        "check_out": content.get("check_out_time", ""),
    })

df_content = pd.DataFrame(content_data)

# Merge with prices from df_hotels
df_full = df_hotels.merge(df_content, on="hid", how="left")
df_full = df_full[["hotel_id", "name", "stars", "kind", "price", "meal", "address"]]
df_full = df_full.sort_values("price", ascending=True).reset_index(drop=True)
df_full.index += 1

df_full

,hotel_id,name,stars,kind,price,meal,address
1,vozle_parka_botanicheskiy_sad_flat,Квартира возле парка Ботанический сад,0,Apartment,16671.0,nomeal,"улица Сельскохозяйственная, д.17к4, Москва"
2,minihotel_ladomir_na_yauze,Отель Ладомир на Яузе,0,Hotel,16729.0,nomeal,"Большой Матросский переулок, д.1 помещ 1/1, Мо..."
3,brusnika_schelkovskaya_minihotel,Мини гостиница Брусника Холдинг,1,Mini-hotel,17489.0,nomeal,"улица Амурская, д.62, Москва"
4,zvezda_hotel_6,Городской отель Звезда,0,Hotel,17500.0,nomeal,"г. Люберцы ул. 3-е Почтовое Отделение, д. 44А,..."
5,knokey_tekstilschiki_apartments,Апартаменты Knokey Текстильщики,0,Apartment,17687.0,nomeal,"улица Грайвороновская, д.4, строение 1, Москва"
...,...,...,...,...,...,...,...
201,palmira_garden_hotel_spa_palmira_garden,Отель СПА Palmira Garden,4,Hotel,96600.0,nomeal,"ул. Школьная 92, Видное"
202,otel_radisson_slavyanskaya_and_business_centre,Отель Radisson Slavyanskaya Hotel & Busi,4,Hotel,97600.0,nomeal,"Площадь Евразии, д.2, Москва"
203,doubletree_by_hilton_moscow_vnukovo_airport,DoubleTree by Hilton Moscow — Vnukovo Ai,4,Hotel,97750.0,nomeal,"улица 2-я Рейсовая, дом 2, Москва"
204,grand_revival_hotel,Отель Revival Hotel,3,Hotel,98647.0,nomeal,"улица Петровка 19 стр.3, Москва"


In [13]:
from services import batch_get_reviews, filter_reviews, HotelReviewsFiltered

print(f"[batch_get_reviews_start] Загрузка отзывов для {len(hotel_hids)} отелей...")
raw_reviews = await batch_get_reviews(client, hotel_hids, LANGUAGE)
reviews_map = filter_reviews(raw_reviews)

total_raw = sum(len(revs) for revs in raw_reviews.values())
total_filtered = sum(len(rd["reviews"]) for rd in reviews_map.values())
total_filtered_by_age = sum(rd["filtered_by_age"] for rd in reviews_map.values())
hotels_with_reviews = len(reviews_map)
total_positive = sum(rd["positive_count"] for rd in reviews_map.values())
total_neutral = sum(rd["neutral_count"] for rd in reviews_map.values())
total_negative = sum(rd["negative_count"] for rd in reviews_map.values())

print(
    f"[batch_get_reviews_done] Всего {hotels_with_reviews} отелей с отзывами из {len(hotel_hids)}"
)
print(
    f"  Обработано {total_raw} отзывов → {total_filtered} релевантных "
    f"(отсечено по давности: {total_filtered_by_age})"
)
print(
    f"  Сегменты: {total_positive} позитивных, {total_neutral} нейтральных, {total_negative} негативных"
)

[batch_get_reviews_start] Загрузка отзывов для 205 отелей...
[batch_get_reviews_done] Всего 205 отелей с отзывами из 205
  Обработано 37371 отзывов → 9482 релевантных (отсечено по давности: 6010)
  Сегменты: 5344 позитивных, 2297 нейтральных, 1841 негативных


In [14]:
# Create DataFrame with reviews summary
reviews_data = []
for hid, data in reviews_map.items():
    hotel_id = next((h["id"] for h in hotels if h["hid"] == hid), "")
    reviews_data.append({
        "hotel_id": hotel_id,
        "hid": hid,
        "total": data["total_reviews"],
        "filtered_by_age": data["filtered_by_age"],
        "positive": data["positive_count"],
        "neutral": data["neutral_count"],
        "negative": data["negative_count"],
    })

df_reviews = pd.DataFrame(reviews_data)
df_reviews = df_reviews.sort_values("total", ascending=False).reset_index(drop=True)
df_reviews.index += 1

# Merge with hotel info
df_reviews_full = df_reviews.merge(
    df_content[["hid", "name", "stars"]],
    on="hid",
    how="left"
)
df_reviews_full = df_reviews_full[["hotel_id", "name", "stars", "total", "filtered_by_age", "positive", "neutral", "negative"]]

# Rating thresholds for display
NEUTRAL_THRESHOLD = 7.0
NEGATIVE_THRESHOLD = 5.0


def show_reviews(hotel_id: str, segment: str = "all", limit: int = 5) -> None:
    """
    Show reviews for a hotel.
    
    Args:
        hotel_id: Hotel ID (e.g. 'rosewood_hong_kong')
        segment: 'positive', 'negative', 'neutral', or 'all'
        limit: Number of reviews to show per segment
    """
    hid = next((h["hid"] for h in hotels if h["id"] == hotel_id), None)
    if not hid:
        print(f"Hotel '{hotel_id}' not found")
        return
    
    data = reviews_map.get(hid)
    if not data:
        print(f"No reviews for hotel '{hotel_id}'")
        return
    
    hotel_name = content_map.get(hid, {}).get("name", hotel_id)
    print(f"{'='*60}")
    print(f"{hotel_name}")
    print(f"Total: {data['total_reviews']} | +{data['positive_count']} / ~{data['neutral_count']} / -{data['negative_count']}")
    print(f"{'='*60}\n")
    
    reviews = data["reviews"]
    
    def print_segment(name: str, filter_fn, limit: int):
        segment_reviews = [r for r in reviews if filter_fn(r)][:limit]
        if not segment_reviews:
            return
        print(f"--- {name} ({len(segment_reviews)}) ---")
        for r in segment_reviews:
            rating = r["rating"]
            date = r["created"][:10]
            lang = r.get("_lang", "?")
            plus = r.get("review_plus", "").strip()
            minus = r.get("review_minus", "").strip()
            print(f"\n[{rating}/10] {date} [{lang}]")
            if plus:
                print(f"  + {plus[:300]}")
            if minus:
                print(f"  - {minus[:300]}")
        print()
    
    if segment in ("all", "positive"):
        print_segment("POSITIVE", lambda r: r["rating"] >= NEUTRAL_THRESHOLD, limit)
    if segment in ("all", "neutral"):
        print_segment("NEUTRAL", lambda r: NEGATIVE_THRESHOLD <= r["rating"] < NEUTRAL_THRESHOLD, limit)
    if segment in ("all", "negative"):
        print_segment("NEGATIVE", lambda r: r["rating"] < NEGATIVE_THRESHOLD, limit)


df_reviews_full

,hotel_id,name,stars,total,filtered_by_age,positive,neutral,negative
0,katyusha_hotel_3,Отель Катюша,3,1186,89,30,30,23
1,hotel_kurortno_razvlekatelny_kompleks_vnukovo_...,Отель Курортно - развлекательный комплек,4,1121,38,30,30,30
2,best_western_vega_hotel,Отель Вега Измайлово,4,1064,533,30,23,18
3,otel_kholidei_inn_moskva_sokolniki,Отель Холидей Инн Москва Сокольники,4,1022,220,30,30,7
4,otel_radisson_slavyanskaya_and_business_centre,Отель Radisson Slavyanskaya Hotel & Busi,4,800,125,30,30,22
...,...,...,...,...,...,...,...,...
200,na_ulitse_novy_arbat_3112_apartments,Апартаменты на улице Новый Арбат 31/12,0,2,0,0,0,2
201,bolshaya_dvukhkomnatnaya_u_metro_dinamo_beloru...,Квартира Большая двухкомнатная у метро Д,0,2,0,2,0,0
202,vmestimostyyu_do_5_gostey_ryadom_s_metro_ot_ts...,Квартира Вместимостью до 5 Гостей Рядом,0,1,0,1,0,0
203,4komnatnye_na_chistykh_prudakh_apartments,Апартаменты 4-комнатные на Чистых прудах,0,1,0,1,0,0


In [15]:
# Example: view reviews for a specific hotel
# show_reviews("four_seasons_st_petersburg")              # all segments, 5 per segment
# show_reviews("four_seasons_st_petersburg", "negative")  # only negative
# show_reviews("four_seasons_st_petersburg", "all", 10)   # all segments, 10 per segment

show_reviews("four_seasons_st_petersburg", limit=3)

Hotel 'four_seasons_st_petersburg' not found


In [16]:
from services import combine_hotels_data, HotelFull

combined = combine_hotels_data(hotels, content_map, reviews_map)
print(f"Combined {len(combined)} hotels with content and reviews")

Combined 205 hotels with content and reviews


In [17]:
import json

from services import estimate_tokens, prepare_hotel_for_llm, presort_hotels, score_hotels

# Estimate tokens before presort
hotels_for_llm_all = [prepare_hotel_for_llm(h) for h in combined]
tokens_before = estimate_tokens(json.dumps(hotels_for_llm_all, ensure_ascii=False))

# Pre-sort by hotel kind tier and prescore, limit to top 100 for LLM scoring
PRESORT_LIMIT = 100
top_hotels = presort_hotels(combined, reviews_map, limit=PRESORT_LIMIT)

# Estimate tokens after presort
hotels_for_llm_top = [prepare_hotel_for_llm(h) for h in top_hotels]
tokens_after = estimate_tokens(json.dumps(hotels_for_llm_top, ensure_ascii=False))

print(f"[presort_done] {len(combined)} отелей → {len(top_hotels)} (лимит {PRESORT_LIMIT})")
print(f"  Токены: ~{tokens_before:,} → ~{tokens_after:,} (экономия {tokens_before - tokens_after:,})")

[presort_done] 205 отелей → 100 (лимит 100)
  Токены: ~291,063 → ~149,870 (экономия 141,193)


In [20]:
scoring_results: list[dict] | None = None
total_tokens_used = 0

async for result in score_hotels(top_hotels, USER_PREFERENCES):
    if result["type"] == "start" and result["start"]:
        start = result["start"]
        print(f"\n[scoring_start] {start['total_hotels']} отелей, {start['total_batches']} батчей (~{start['estimated_tokens']:,} токенов)")
    elif result["type"] == "batch_start" and result["batch_start"]:
        bs = result["batch_start"]
        print(f"  → Batch {bs['batch']}/{bs['total_batches']}: {bs['hotels_in_batch']} отелей, ~{bs['estimated_tokens']:,} токенов")
    elif result["type"] == "retry" and result["retry"]:
        retry = result["retry"]
        print(f"    ⚠ Retry {retry['attempt']}/{retry['max_attempts']}")
    elif result["type"] == "progress" and result["progress"]:
        progress = result["progress"]
        print(f"  ✓ Batch done ({progress['processed']}/{progress['total']})")
    elif result["type"] == "error" and result["error"]:
        error = result["error"]
        print(f"\n  ❌ ERROR [{error['error_type']}]: {error['message']}")
        break
    elif result["type"] == "done":
        scoring_results = result["results"]

if scoring_results is None:
    print("\n[scoring_failed]")
else:
    print(f"\n[scoring_done] {len(scoring_results)} отелей оценено")


[scoring_start] 100 отелей, 4 батчей (~152,294 токенов)
  → Batch 1/4: 25 отелей, ~38,851 токенов
  ✓ Batch done (25/100)
  → Batch 2/4: 25 отелей, ~37,718 токенов
  ✓ Batch done (50/100)
  → Batch 3/4: 25 отелей, ~36,689 токенов
  ✓ Batch done (75/100)
  → Batch 4/4: 25 отелей, ~39,035 токенов
  ✓ Batch done (100/100)

[scoring_done] 93 отелей оценено


In [21]:
from typing import Any


def display_top_hotels(
    results: list[dict[str, Any]],
    hotels_data: list[dict[str, Any]],
    top_n: int = 10,
) -> pd.DataFrame:
    """Display top N scored hotels with details and Ostrovok links."""
    # Build lookup maps
    name_map = {h.get("id", ""): h.get("name", h.get("id", "")) for h in hotels_data}
    hid_map = {h.get("id", ""): h.get("hid", 0) for h in hotels_data}
    
    print(f"\n{'='*80}")
    print(f"TOP {top_n} HOTELS")
    print(f"{'='*80}\n")

    data = []
    for i, hotel in enumerate(results[:top_n], 1):
        hotel_id = hotel.get("hotel_id", "")
        score = hotel.get("score", 0)
        name = name_map.get(hotel_id, hotel_id)
        hid = hid_map.get(hotel_id, 0)
        reasons = hotel.get("top_reasons", [])
        penalties = hotel.get("score_penalties", [])
        
        # Generate Ostrovok URL
        url = ostrovok_url(
            hotel_id=hotel_id,
            hid=hid,
            checkin=CHECKIN_DATE,
            checkout=CHECKOUT_DATE,
            guests=GUESTS,
            region_id=region_id,
        )

        # Print detailed info
        print(f"{i}. {name}")
        print(f"   Score: {score}/100")
        if reasons:
            print(f"   + {'; '.join(reasons[:3])}")
        if penalties:
            print(f"   - {'; '.join(penalties[:5])}")
        print(f"   🔗 {url}")
        print()
        
        # Collect for DataFrame
        data.append({
            "name": name[:40],
            "score": score,
            "reasons": "; ".join(reasons[:2])[:80] if reasons else "",
            "url": url,
        })
    
    df = pd.DataFrame(data)
    df.index = range(1, len(df) + 1)
    total_found = len(hotels_data)
    selected = min(top_n, len(results))
    print(f"Всего найдено {total_found} отелей на эти даты. ")
    print(f"Подобраны лучшие {selected} по вашим критериям.")
    return df


pd.set_option("display.max_colwidth", 100)
display_top_hotels(scoring_results, combined, top_n=10)


TOP 10 HOTELS

1. Отель Веллион Водный
   Score: 95/100
   + Трехкомнатные апартаменты гарантируют две комнаты и две кровати; Высокие оценки за чистоту и комфорт; Рядом с метро и торговым центром
   - Небольшое замечание по качеству уборки в одном из отзывов
   🔗 https://ostrovok.ru/hotel/russia/moscow/mid8854273/wellion_vodny_hotel/?dates=30.01.2026-04.02.2026&guests=4&q=2395

2. Гостиница Первомайская
   Score: 95/100
   + Есть семейный номер с 2 комнатами; Высокие оценки за чистоту и уют; Отличные отзывы о персонале и завтраках
   - Расположение далеко от центра (15 км)
   🔗 https://ostrovok.ru/hotel/russia/moscow/mid7668742/gostinitsa_pervomaiskaia_2/?dates=30.01.2026-04.02.2026&guests=4&q=2395

3. Отель Хитровка
   Score: 95/100
   + Есть семейные номера с 2 комнатами; Подтвержденное наличие двухкомнатных номеров в отзывах; Хорошие отзывы о комфорте и расположении
   - Некоторые отзывы упоминают необходимость ремонта и проблемы с сантехникой
   🔗 https://ostrovok.ru/hotel/russia/

,name,score,reasons,url
1,Отель Веллион Водный,95,Трехкомнатные апартаменты гарантируют две комнаты и две кровати; Высокие оценки,https://ostrovok.ru/hotel/russia/moscow/mid8854273/wellion_vodny_hotel/?dates=30.01.2026-04.02.2...
2,Гостиница Первомайская,95,Есть семейный номер с 2 комнатами; Высокие оценки за чистоту и уют,https://ostrovok.ru/hotel/russia/moscow/mid7668742/gostinitsa_pervomaiskaia_2/?dates=30.01.2026-...
3,Отель Хитровка,95,Есть семейные номера с 2 комнатами; Подтвержденное наличие двухкомнатных номеров,https://ostrovok.ru/hotel/russia/moscow/mid7660730/hitrovka_hotel_2/?dates=30.01.2026-04.02.2026...
4,Отель Radisson Slavyanskaya Hotel & Busi,95,Гарантированный двухкомнатный люкс; Наличие двух спальных мест (кровать + диван),https://ostrovok.ru/hotel/russia/moscow/mid7467357/otel_radisson_slavyanskaya_and_business_centr...
5,Отель Лесная Сафмар (бывший Холидей Инн,94,Двухкомнатный люкс в центре Москвы; Два спальных места (кровать и диван),https://ostrovok.ru/hotel/russia/moscow/mid7467380/kholidei_inn_moskva_lesnaia/?dates=30.01.2026...
6,Отель Кунь Лунь,92,Двухкомнатные люксы с большой площадью; Очень просторные номера и удобные кроват,https://ostrovok.ru/hotel/russia/moscow/mid10013909/kunlun/?dates=30.01.2026-04.02.2026&guests=4...
7,Отель Edge Seligerskaya Moscow,92,Наличие четырехместных люксов с 2 комнатами; Хорошие отзывы о чистоте и сервисе,https://ostrovok.ru/hotel/russia/moscow/mid7600189/iris_congress_hotel/?dates=30.01.2026-04.02.2...
8,Отель Багратион,92,Наличие двухкомнатного семейного люкса; Отличные отзывы о чистоте и персонале,https://ostrovok.ru/hotel/russia/moscow/mid7852775/bagration_hotel/?dates=30.01.2026-04.02.2026&...
9,Отель Hampton by Hilton Moscow Рогожский,92,Четко указан двухкомнатный семейный номер; Наличие двух спальных мест (King size,https://ostrovok.ru/hotel/russia/moscow/mid9770069/hampton_by_hilton_moscow_rogozhsky_val/?dates...
10,Отель Mamaison All-Suites Spa Pokrovka M,90,Просторные двухкомнатные апартаменты в центре; Пятизвездочный сервис и отличный,https://ostrovok.ru/hotel/russia/moscow/mid7590428/mamaison_allsuites_spa_hotel_pokrovka/?dates=...


In [22]:
from services import finalize_scored_hotels, summarize_results

# Finalize scored hotels (merge hotel data with scoring results)
scored_hotels = finalize_scored_hotels(top_hotels, scoring_results)

# Generate AI summary of search results
print("[summary_start] Generating AI summary of top 10 results...")
summary_result = await summarize_results(scored_hotels, USER_PREFERENCES)

if summary_result["error"]:
    print(f"[summary_error] {summary_result['error']}")
else:
    summary = summary_result["summary"]
    print("[summary_done]\n")
    print("=" * 80)
    print("AI SUMMARY")
    print("=" * 80)
    print(f"\n📋 Overview:\n{summary.overview}\n")
    print("🏆 Top Picks:")
    for i, pick in enumerate(summary.top_picks, 1):
        print(f"  {i}. {pick.hotel_name}")
        print(f"     {pick.why_recommended}")
    print(f"\n⚖️ Considerations:\n{summary.considerations}")
    print(f"\n💡 Final Advice:\n{summary.final_advice}")

[summary_start] Generating AI summary of top 10 results...
[summary_done]

AI SUMMARY

📋 Overview:
Поиск прошел успешно: найдено несколько отличных вариантов с двухкомнатными номерами и высокими оценками за чистоту. Большинство отелей предлагают семейные люксы или апартаменты, которые полностью соответствуют вашим требованиям.

🏆 Top Picks:
  1. Отель Веллион Водный
     Предлагает трехкомнатные апартаменты, что гарантирует наличие двух отдельных комнат и нескольких кроватей. Имеет высокие оценки за чистоту.
  2. Гостиница Первомайская
     Специализируется на семейных двухкомнатных номерах с отличными отзывами о чистоте и персонале.
  3. Отель Хитровка
     Расположен в центре и предлагает подтвержденные в отзывах двухкомнатные семейные номера с хорошим уровнем комфорта.
  4. Отель Багратион
     Тихий отель в центре с двухкомнатными семейными люксами и безупречной репутацией в плане чистоты.

⚖️ Considerations:
Многие двухкомнатные номера в подборке предлагают комбинацию «кровать + д